In [1]:
import os
import awkward as ak

input_dir = "/pscratch/sd/c/chlcheng/dataset/anomaly_detection/LHC_Olympics_2020/"
input_path = os.path.join(input_dir, "point_cloud_features_signal_plus_background.parquet")

In [2]:
from typing import Dict, List, Optional
import numpy as np
import copy

from quickstats import AbstractObject

class PointCloudDataset(AbstractObject):
    """
    This loader will load data in memory. Improvements to be made to load lazily.
    """
    DEFAULT_FEATURE_DICT = {
        "point_coords"   : ["part_delta_eta", "part_delta_phi"],
        "point_features" : ["part_pt", "part_delta_eta", "part_delta_phi", "part_delta_R"],
        "jet_features"   : ["jet_pt", "jet_eta", "jet_phi", "jet_m", "N", "tau12", "tau23"]
    }

    DEFAULT_LABEL_MAP = {
        "background" : 0,
        "signal"     : 1
    }

    DEFAULT_LABEL_FRACTION = {
        "background" : 1,
        "signal"     : 1
    }
    
    def __init__(self, filename:str,
                 sample_sizes:Optional[Dict]=None,
                 label_map:Optional[Dict]=None,
                 label_fraction:Optional[Dict]=None,
                 feature_dict:Optional[Dict]=None,
                 num_jets:int=2, pad_size:int=300,
                 shuffle:bool=True, seed:int=2023,
                 verbosity:str="INFO"):
        super().__init__(verbosity=verbosity)
        if feature_dict is None:
            self.feature_dict = copy.deepcopy(self.DEFAULT_FEATURE_DICT)
        else:
            self.feature_dict = copy.deepcopy(feature_dict)
        if label_map is None:
            self.label_map = copy.deepcopy(self.DEFAULT_LABEL_MAP)
        else:
            self.label_map = copy.deepcopy(label_map)
        if label_fraction is None:
            self.label_fraction = copy.deepcopy(self.DEFAULT_LABEL_FRACTION)
        else:
            self.label_fraction = copy.deepcopy(label_fraction)
        if sample_sizes is None:
            sample_sizes = {label: None for label in self.label_map}
        else:
            sample_sizes = {label: sample_sizes.get(label, None) for label in self.label_map}
        self.num_jets = num_jets
        self.pad_size  = pad_size
        self.sample_sizes = sample_sizes
        self.shuffle = shuffle
        self.seed = seed
        self._features = {}
        self._labels   = None
        self.load(filename)

    def __len__(self):
        return self._labels.shape[0]
        
    @staticmethod
    def is_ragged(array):
        return isinstance(array.type.content, ak.types.ListType)

    @staticmethod
    def get_padded_array(array, pad_size:int, clip:bool=True, pad_val:float=0,
                         sample_size:Optional[int]=None):
        if sample_size is None:
            return ak.to_numpy(ak.fill_none(ak.pad_none(array, pad_size, clip=clip), pad_val))
        return ak.to_numpy(ak.fill_none(ak.pad_none(array[:sample_size], pad_size, clip=clip), pad_val))

    @staticmethod
    def get_array(array, sample_size:Optional[int]=None):
        if sample_size is None:
            return ak.to_numpy(array)
        return ak.to_numpy(array[:sample_size])
                  
    @staticmethod
    def get_mask_array(array, pad_size:int, clip:bool=True, sample_size:Optional[int]=None):
        if sample_size is None:
            return ak.to_numpy(ak.pad_none(array, pad_size, clip=clip)).mask
        return ak.to_numpy(ak.pad_none(array[:sample_size], pad_size, clip=clip)).mask
        
    @staticmethod
    def get_partial_array(array, fraction:float=1, shuffle:bool=True):
        size = array.shape[0]
        new_size = round(size * fraction)
        if shuffle:
            index = np.arange(new_size, dtype=int)
            np.random.shuffle(index)
            return array[index]
        if fraction == 1:
            return array
        return array[:new_size]

    def load(self, filename:str):
        self.stdout.info(f'Loading dataset from "{filename}"')
        ak_arrays = ak.from_parquet(filename)
        features = {}
        labels = []
        masks  = []
        np.random.seed(self.seed)
        for label, label_val in self.label_map.items():
            self.stdout.info(f'Preparing data for the class "{label}"')
            if label not in ak_arrays.fields:
                raise RuntimeError(f'dataset does not contain data with label "{label}"')
            label_arrays = ak_arrays[label]
            label_size = len(label_arrays)
            sample_size = self.sample_sizes[label]
            if (sample_size is not None):
                if sample_size > label_size:
                    raise RuntimeError('can not request more events than is available for the '
                                       f'class "{label}" (requested = {sample_size}, available = {label_size}')
                label_size = sample_size
            self.stdout.info(f'Size of class data: {label_size}')
            label_fraction = self.label_fraction.get(label, 1)
            if label_fraction != 1:
                label_size = round(label_size * label_fraction)
            labels.append(np.full(label_size, label_val))
            mask_arrays = None
            for feature_type in self.feature_dict:
                self.stdout.info(f'Working on feature type "{feature_type}"')
                if feature_type not in features:
                    features[feature_type] = []
                columns = self.feature_dict[feature_type]
                jet_feature_arrays = []
                jet_mask_arrays = []
                for i in range(1, self.num_jets + 1):
                    self.stdout.info(f'Jet index: {i}')
                    jet_key = f'j{i}'
                    feature_arrays = []
                    for column in columns:
                        self.stdout.info(f'Loading data for the feature "{column}"')
                        arrays = label_arrays[jet_key][column]
                        if self.is_ragged(arrays):
                            # only get the mask once per jet
                            if (mask_arrays is None) and (len(jet_mask_arrays) == i - 1):
                                mask_array = self.get_mask_array(arrays, self.pad_size,
                                                                 sample_size=sample_size)
                                jet_mask_arrays.append(mask_array)
                            arrays = self.get_padded_array(arrays, self.pad_size,
                                                           sample_size=sample_size)
                        else:
                            arrays = self.get_array(arrays, sample_size=sample_size)
                        feature_arrays.append(arrays)
                    # shape = (nevent, nparticles, nfeatuers)
                    feature_arrays = self.get_partial_array(np.stack(feature_arrays, -1),
                                                            label_fraction,
                                                            shuffle=self.shuffle)
                    jet_feature_arrays.append(feature_arrays)
                # shape = (nevent, njet, nparticles, nfeatuers)
                jet_feature_arrays = np.stack(jet_feature_arrays, axis=1)
                features[feature_type].append(jet_feature_arrays)
                if jet_mask_arrays:
                    mask_arrays = np.stack(jet_mask_arrays, axis=1)
            if mask_arrays is not None:
                masks.append(mask_arrays)
        self.stdout.info(f'Combining data from all classes')
        sizes = []
        for feature_type in features:
            features[feature_type] = np.concatenate(features[feature_type])
            sizes.append(features[feature_type].shape[0])
        labels = np.concatenate(labels)
        if len(set(sizes)) != 1:
            raise RuntimeError("inconsistent sample size in different feature types")
        if sizes[0] != labels.shape[0]:
            raise RuntimeError("inconsistent sample size between features and labels")
        if masks:
            masks  = np.concatenate(masks)
            assert masks.shape[0] == sizes[0]
        else:
            masks  = None
        if self.shuffle:
            self.stdout.info(f"Shuffling events (size = {sizes[0]})")
            index = np.arange(sizes[0])
            np.random.shuffle(index)
            for feature_type in features:
                features[feature_type] = features[feature_type][index]
            labels = labels[index]
            if masks is not None:
                masks = masks[index]
        self._features = features
        self._labels = labels
        self._masks  = masks

    @property
    def x(self):
        return self._features
    
    @property
    def y(self):
        return self._labels

    @property
    def masks(self):
        return self._masks

In [3]:
sample_sizes = {
    "background": 1_000,
    "signal"    :  1_00
}
dataset = PointCloudDataset(input_path, sample_sizes=sample_sizes)

[INFO] Loading dataset from "/pscratch/sd/c/chlcheng/dataset/anomaly_detection/LHC_Olympics_2020/point_cloud_features_signal_plus_background.parquet"
[INFO] Preparing data for the class "background"
[INFO] Size of class data: 1000
[INFO] Working on feature type "point_coords"
[INFO] Jet index: 1
[INFO] Loading data for the feature "part_delta_eta"
[INFO] Loading data for the feature "part_delta_phi"
[INFO] Jet index: 2
[INFO] Loading data for the feature "part_delta_eta"
[INFO] Loading data for the feature "part_delta_phi"
[INFO] Working on feature type "point_features"
[INFO] Jet index: 1
[INFO] Loading data for the feature "part_pt"
[INFO] Loading data for the feature "part_delta_eta"
[INFO] Loading data for the feature "part_delta_phi"
[INFO] Loading data for the feature "part_delta_R"
[INFO] Jet index: 2
[INFO] Loading data for the feature "part_pt"
[INFO] Loading data for the feature "part_delta_eta"
[INFO] Loading data for the feature "part_delta_phi"
[INFO] Loading data for the 

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization

2023-10-18 11:27:58.771728: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
points       = dataset.x['point_coords']
features     = dataset.x['point_features']
jet_features = dataset.x['jet_features']
masks        = dataset.masks

In [6]:
# (nevent, njet, nparticle, ncoords)
points.shape

(1100, 2, 300, 2)

In [7]:
# (nevent, njet, nparticle, nfeatures)
features.shape

(1100, 2, 300, 4)

In [8]:
# (nevent, njet, njetfeatures)
jet_features.shape

(1100, 2, 7)

In [37]:
def get_distance_matrix(A, B):
    # A and B has shape (nevent, njet, nparticle, ncoords)
    # D has shape (nevent, njet, nparticle, nparticle)
    # i.e. the 3rd and 4th dimensions represent the distance between the i-th and j-th particles
    with tf.name_scope("DMatrix"):
        r_A = tf.reduce_sum(A * A, axis=3, keepdims=True)
        r_B = tf.reduce_sum(B * B, axis=3, keepdims=True)
        m = tf.matmul(A, tf.transpose(B, perm=(0, 1, 3, 2)))
        D = r_A - 2 * m + tf.transpose(r_B, perm=(0, 1, 3, 2))
        return D

def KNN(features, nn_indices, num_points:int=300, num_jets:int=2, K:int=16):
    """
    features: Tensor of shape (nevent, njet, nparticle, nfeature)
        Features of the point cloud particles
    nn_indices: Tensor of shape (nevent, njet, nparticle, nneighbor)
        Indices of the nearest-neighbors of the point cloud particles
    num_points: int
        Number of partcles in the point cloud
    num_jets: int
        Number of jets in an event
    K : int
        Number of nearest-neighbors
    nn: tensor of shape (nevent, njet, nparticle, k)
        Indices of the nearest-neighbors
    """
    with tf.name_scope("KNN"):
        feature_shape = tf.shape(features)
        batchsize = feature_shape[0]
        num_features = feature_shape[-1]
        batch_indices = tf.tile(tf.reshape(tf.range(batchsize), (-1, 1, 1, 1, 1)),
                                (1, num_jets, num_points, K, 1))
        indices = tf.concat([batch_indices, tf.expand_dims(nn_indices, axis=4)], axis=4)
        rs_indices = tf.reshape(indices, (batchsize * num_jets, num_points, K, 2))
        rs_features      = tf.reshape(features, (batchsize * num_jets, num_points, num_features))
        # shape = (nevent, njet, nparticle, nneighbor, nfeature)
        return tf.reshape(tf.gather_nd(rs_features, rs_indices), (batchsize, num_jets, num_points, K, num_features))

In [10]:
D = get_distance_matrix(points, points)

2023-10-18 11:28:11.017494: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [11]:
K = 16
_, nn_indices = tf.nn.top_k(-D, k = K + 1)
# remove the first index which is always the particle itself
nn_indices = nn_indices[:, :, :, 1:]

In [38]:
KNN_fts = KNN(features, nn_indices)

In [7]:
KNN_fts.shape

In [8]:
coord_shift = tf.multiply(999., tf.cast(masks, dtype='float32'))

2023-10-18 01:40:49.800834: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
conv_params = [
    (16, (64, 64, 64)),
    (16, (128, 128, 128)),
    (16, (256, 256, 256)),
]

In [ ]:
from quickstats import semistaticmethod, AbstractObject

import tensorflow as tf

class ModifiedParticleNet(AbstractObject):
    
    def __init__(self,
                 verbosity:str="INFO"):
        super().__init__(verbosity=verbosity)):

    @staticmethod
    def get_distance_matrix(A, B):
        # A and B has shape (nevent, njet, nparticle, ncoords)
        # D has shape (nevent, njet, nparticle, nparticle)
        # i.e. the 3rd and 4th dimensions represent the distance between the i-th and j-th particles
        with tf.name_scope("DMatrix"):
            r_A = tf.reduce_sum(A * A, axis=3, keepdims=True)
            r_B = tf.reduce_sum(B * B, axis=3, keepdims=True)
            m = tf.matmul(A, tf.transpose(B, perm=(0, 1, 3, 2)))
            D = r_A - 2 * m + tf.transpose(r_B, perm=(0, 1, 3, 2))
            return D

    @staticmethod
    def KNN(features, nn_indices, num_points:int=300, num_jets:int=2, K:int=16):
        """
        features: Tensor of shape (nevent, njet, nparticle, nfeature)
            Features of the point cloud particles
        nn_indices: Tensor of shape (nevent, njet, nparticle, nneighbor)
            Indices of the nearest-neighbors of the point cloud particles
        num_points: int
            Number of partcles in the point cloud
        num_jets: int
            Number of jets in an event
        K : int
            Number of nearest-neighbors
        nn: tensor of shape (nevent, njet, nparticle, k)
            Indices of the nearest-neighbors
        """
        with tf.name_scope("KNN"):
            feature_shape = tf.shape(features)
            batchsize = feature_shape[0]
            num_features = feature_shape[-1]
            batch_indices = tf.tile(tf.reshape(tf.range(batchsize), (-1, 1, 1, 1, 1)),
                                    (1, num_jets, num_points, K, 1))
            indices = tf.concat([batch_indices, tf.expand_dims(nn_indices, axis=4)], axis=4)
            rs_indices = tf.reshape(indices, (batchsize * num_jets, num_points, K, 2))
            rs_features      = tf.reshape(features, (batchsize * num_jets, num_points, num_features))
            # shape = (nevent, njet, nparticle, nneighbor, nfeature)
            return tf.reshape(tf.gather_nd(rs_features, rs_indices), (batchsize, num_jets, num_points, K, num_features))
            
    @semistaticmethod
    def EdgeConv(self, points, features,  channels,
                 num_jets:int=2,
                 num_points:int=300,
                 K:int=16,
                 with_bn:bool=True,
                 activation:str='relu',
                 pooling:str='average'):
        """
            points: Tensor of shape (nevent, njet, nparticle, ncoords)
                Coordinates of the point cloud particles
            features: Tensor of shape (nevent, njet, nparticle, nfeatures)
                Features of the point cloud particles
            num_points: int
                Number of partcles in the point cloud
            num_jets: int
                Number of jets in an event
            K: int
                Number of nearest-neighbors
        """
        with tf.name_scope('EdgeConv'):
            D = self.get_distance_matrix(points, points)
            # get indices of nearest neighbors
            _, indices = tf.nn.top_k(-D, k = K + 1)
            # remove the first index which is always the particle itself
            indices = indices[:, :, :, 1:]

            knn_fts = self.KNN(features, indices, num_points=num_points,
                               num_jets=num_jets, K=K)
            >>>>>>>
            knn_fts_center = tf.tile(tf.expand_dims(features, axis=2), (1, 1, K, 1))  # (N, P, K, C)
            >>>>>>>
            knn_fts = tf.concat([knn_fts_center, tf.subtract(knn_fts, knn_fts_center)], axis=-1)  # (N, P, K, 2*C)
    
            x = knn_fts
            for idx, channel in enumerate(channels):
                x = keras.layers.Conv2D(channel, kernel_size=(1, 1), strides=1, data_format='channels_last',
                                        use_bias=False if with_bn else True, kernel_initializer='glorot_normal', name='%s_conv%d' % (name, idx))(x)
                if with_bn:
                    x = keras.layers.BatchNormalization(name='%s_bn%d' % (name, idx))(x)
                if activation:
                    x = keras.layers.Activation(activation, name='%s_act%d' % (name, idx))(x)
            >>>>>>>
            if pooling == 'max':
                fts = tf.reduce_max(x, axis=2)  # (N, P, C')
            else:
                fts = tf.reduce_mean(x, axis=2)  # (N, P, C')
            >>>>>>>
            # shortcut
            sc = keras.layers.Conv2D(channels[-1], kernel_size=(1, 1), strides=1, data_format='channels_last',
                                     use_bias=False if with_bn else True, kernel_initializer='glorot_normal', name='%s_sc_conv' % name)(tf.expand_dims(features, axis=2))
            if with_bn:
                sc = keras.layers.BatchNormalization(name='%s_sc_bn' % name)(sc)
            sc = tf.squeeze(sc, axis=2)
    
            if activation:
                return keras.layers.Activation(activation, name='%s_sc_act' % name)(sc + fts)  # (N, P, C')
            else:
                return sc + fts

    
    def get_model(points, features=None, mask=None, setting=None, name='particle_net'):
        # points : (N, P, C_coord)
        # features:  (N, P, C_features), optional
        # mask: (N, P, 1), optinal
    
        with tf.name_scope(name):
            if features is None:
                features = points
    
            if mask is not None:
                mask = tf.cast(tf.not_equal(mask, 0), dtype='float32')  # 1 if valid
                coord_shift = tf.multiply(999., tf.cast(tf.equal(mask, 0), dtype='float32'))  # make non-valid positions to 99
    
            fts = tf.squeeze(keras.layers.BatchNormalization(name='%s_fts_bn' % name)(tf.expand_dims(features, axis=2)), axis=2)
            for layer_idx, layer_param in enumerate(setting.conv_params):
                K, channels = layer_param
                pts = tf.add(coord_shift, points) if layer_idx == 0 else tf.add(coord_shift, fts)
                fts = edge_conv(pts, fts, setting.num_points, K, channels, with_bn=True, activation='relu',
                                pooling=setting.conv_pooling, name='%s_%s%d' % (name, 'EdgeConv', layer_idx))
    
            if mask is not None:
                fts = tf.multiply(fts, mask)
    
            pool = tf.reduce_mean(fts, axis=1)  # (N, C)
    
            if setting.fc_params is not None:
                x = pool
                for layer_idx, layer_param in enumerate(setting.fc_params):
                    units, drop_rate = layer_param
                    x = keras.layers.Dense(units, activation='relu')(x)
                    if drop_rate is not None and drop_rate > 0:
                        x = keras.layers.Dropout(drop_rate)(x)
                out = keras.layers.Dense(setting.num_class, activation='softmax')(x)
                return out  # (N, num_classes)
            else:
                return pool

        >>>
            setting = _DotDict()
            setting.num_class = num_classes
            # conv_params: list of tuple in the format (K, (C1, C2, C3))
            setting.conv_params = [
                (16, (64, 64, 64)),
                (16, (128, 128, 128)),
                (16, (256, 256, 256)),
                ]
            # conv_pooling: 'average' or 'max'
            setting.conv_pooling = 'average'
            # fc_params: list of tuples in the format (C, drop_rate)
            setting.fc_params = [(256, 0.1)]
            setting.num_points = input_shapes['points'][0]
        
            points = keras.Input(name='points', shape=input_shapes['points'])
            features = keras.Input(name='features', shape=input_shapes['features']) if 'features' in input_shapes else None
            mask = keras.Input(name='mask', shape=input_shapes['mask']) if 'mask' in input_shapes else None
            outputs = _particle_net_base(points, features, mask, setting, name='ParticleNet')
        
            return keras.Model(inputs=[points, features, mask], outputs=outputs, name='ParticleNet')        
        >>>